import pandas as pd
# ^^^ pyforest auto-imports - don't write above this line
# Finding the Coordinates of Players Birthplaces

# Imports 

In [ ]:
import pandas as pd
from functions import *
from geopandas_functions import *
%run functions.py
import geopy
from geopy.geocoders import Nominatim

In [ ]:
player_info_df = pd.read_csv("data/player_info_df.csv")
player_info_df.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
n_am_players = player_info_df[player_info_df['nationality'].isin(['CAN', 'USA'])].reset_index(drop=True)

# Using Geopy

In [ ]:
# initializing geopy
geolocator = Nominatim(user_agent="NHL_birthplace_analysis")

In [ ]:
# the locations that I will feed into geopy
n_am_players['location'] = n_am_players['birthCity'] + "," + n_am_players['birthStateProvince']

In [ ]:
# running the function to find all locations
# this took slightly long but was still faster than using wikipedia
lat = []
lng = []
for item in n_am_players['location']:
    try:
        location = geolocator.geocode(item)
        lat.append(location.latitude)
        lng.append(location.longitude)
    except AttributeError:
        lat.append(None)
        lng.append(None)

In [ ]:
# checking that I have a value (even if None) for each value in n_am_players
n_am_players.shape[0] == len(lat)

In [ ]:
# so only 2 values couldn't be found 
lat.count(None)

## Correcting the errors


In [ ]:
# player 1
index_1 = lat.index(None)
n_am_players.iloc[index_1]['location']

In [ ]:
# This helps me to turn the coordinates I got from google into a string
test_str = turn_coords_to_correct_format("45.4033° N, 73.9504° W")
test_str

In [ ]:
# player 2
index_2 = lat.index(None, index_1+1)
n_am_players.iloc[index_2]['location']

In [ ]:
# This helps me to turn the coordinates I got from google into a string
test_str = turn_coords_to_correct_format("32.2163° N, 80.7526° W")
test_str

In [ ]:
# adding those values to the coordinates lists
lat[index_1] = 45.4033
lat[index_2] = 32.2163
lng[index_1] = -73.9504
lng[index_2] = -80.7526

## Adding Data to the dataframe

In [ ]:
n_am_players['lat'] = lat
n_am_players['lng'] = lng

# Saving Data

In [ ]:
n_am_players.to_csv("data/n_am_players_w_coordinates.csv")